In [172]:
import pandas as pd

In [173]:
# 读入数据
fmct = pd.read_excel('FMCT.xlsx')
fmop = pd.read_excel('FMOP.xlsx')
fmvs = pd.read_excel('FMVS.xlsx')
fmpg = pd.read_excel('FMPG.xlsx', dtype=str)
fmpg = fmpg.replace('nan', '')

In [174]:
# 重命名列名
nfmvs = pd.DataFrame(fmvs[['VSAPPL', 'VSNUMDOC', 'VSTITRE']])
nfmct = pd.DataFrame(fmct[['CTAPPL', 'CTNUMDOC', 'CTTITRE']])
nfmop = pd.DataFrame(fmop[['OPAPPL', 'OPNUMDOC', 'OPTITRE']])
nfmvs.rename(columns={'VSAPPL': 'PROTOCOL', 'VSNUMDOC': 'REFERENCE', 'VSTITRE': 'TITLE'}, inplace=True)
nfmct.rename(columns={'CTAPPL': 'PROTOCOL', 'CTNUMDOC': 'REFERENCE', 'CTTITRE': 'TITLE'}, inplace=True)
nfmop.rename(columns={'OPAPPL': 'PROTOCOL', 'OPNUMDOC': 'REFERENCE', 'OPTITRE': 'TITLE'}, inplace=True)
docs = [nfmvs, nfmct, nfmop]

In [175]:
fmpg.PGNUMAP = fmpg.PGNUMAP.apply(lambda x: 'B-' + x)

In [176]:
# 日期格式化
def series_element_join(years, months, days):
    ll = []
    for y, m, d in zip(years, months, days):
        ll.append(y + m + d)
#         if y != '' and m != '' and d != '':
#             ll.append(''.join([y, m, d]))
#         else:
#             ll.append('')
    return pd.Series(ll)

In [178]:
# sel_fmpg = fmpg[['PGNUMAP', 'PGCODTE', 'PGAPPL', 'PGNUMDOC', 'PGSIT', 
#       'PGAPPA4', 'PGAPPMM', 'PGAPPJJ', 'PGAPPDOC1', 'PGAPPDOC2',
#      'PGPRVA4', 'PGPRVMM', 'PGPRVJJ', 'PGPRVH', 'PGPRVC', 'PGPRVDOC1', 'PGPRVDOC2']]

# d = {'AC': str(fmpg['PGNUMAP']).str.strip(),
#      'PROTOCOL': fmpg['PGAPPL'].str.strip(),
#      'PGCODTE': fmpg['PGCODTE'],
#      'REFERENCE': fmpg['PGNUMDOC'].str.strip(),
#      'STATUS': fmpg['PGSIT'].str.strip(),
#      'PERFORMACE_DOC': fmpg['PGAPPDOC1'].str.strip() + fmpg['PGAPPDOC2'].str.strip(),
#      'PERFORMACE_DATE': series_element_join(fmpg['PGAPPA4'].str.strip(), fmpg['PGAPPMM'].str.strip(), fmpg['PGAPPJJ'].str.strip()),
#      'FH_DEADLINE': fmpg['PGPRVH'],
#      'CY_DEADLINE': fmpg['PGPRVC'],
#      'DEADLINE_DOC': fmpg['PGPRVDOC1'].str.strip() + fmpg['PGPRVDOC2'].str.strip(),
#      'DEADLINE_DATE': series_element_join(fmpg['PGPRVA4'].str.strip(), fmpg['PGPRVMM'].str.strip(), fmpg['PGPRVJJ'].str.strip())
#     }
# 按导出列名重构fmpg
d = {'AC': fmpg['PGNUMAP'].str.strip(),
     'PROTOCOL': fmpg['PGAPPL'].str.strip(),
     'PGCODTE': fmpg['PGCODTE'],
     'REFERENCE': fmpg['PGNUMDOC'].str.strip(),
     'STATUS': fmpg['PGSIT'].str.strip(),
     'PERFORMACE_DOC': fmpg['PGAPPDOC1'].str.strip() + fmpg['PGAPPDOC2'].str.strip(),
     'PERFORMACE_DATE': series_element_join(fmpg['PGAPPA4'].str.strip(), fmpg['PGAPPMM'].str.strip(), fmpg['PGAPPJJ'].str.strip()),
     'FH_DEADLINE': fmpg['PGPRVH'],
     'CY_DEADLINE': fmpg['PGPRVC'],
     'DEADLINE_DOC': fmpg['PGPRVDOC1'].str.strip() + fmpg['PGPRVDOC2'].str.strip(),
     'DEADLINE_DATE': series_element_join(fmpg['PGPRVA4'].str.strip(), fmpg['PGPRVMM'].str.strip(), fmpg['PGPRVJJ'].str.strip())
    }

nfmpg = pd.DataFrame(d, columns=['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'STATUS', 'PERFORMACE_DOC', 'PERFORMACE_DATE', 
                              'FH_DEADLINE', 'CY_DEADLINE', 'DEADLINE_DOC', 'DEADLINE_DATE'])

In [179]:
# 根据doc类型分组
groupbycode = nfmpg.groupby(nfmpg['PGCODTE'])
dfs = []
for k, v in groupbycode:
    dfs.append(pd.DataFrame(v))

In [180]:
# res = dfs[0].merge(docs[0], 
#              left_on=['PROTOCOL', 'REFERENCE'], 
#              right_on=list(docs[0].columns[0:2]),
#              how='left')

# 根据REFERENCE获取TITLE
mindex = ['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'TITLE', 'STATUS', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE']
merged_dfs = []
for index, df in enumerate(dfs):
    docdf = docs[index]
    merged_dfs.append(pd.merge(dfs[index], docdf, on=['PROTOCOL', 'REFERENCE'], how='left')[mindex])

In [170]:
len(merged_dfs)

1

In [147]:
# all_df = pd.concat(merged_dfs)
# all_df

# 只取job card类型的记录
all_df = merged_dfs[1]

In [148]:
groupbyac = all_df.groupby(['AC'])
ac_dfs = {}
for k, v in groupbyac:
    ac_dfs[k] = pd.DataFrame(v)

In [150]:
# for k, v in ac_dfs.items():
#     print(k)
# len(ac_dfs.items())

1820


In [151]:
output_df = ac_dfs['1820']
output_df[output_df['STATUS'] == 'O'].to_excel('1820.xlsx', columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [47]:
ac_dfs['300A']

,AC,PROTOCOL,PGCODTE,REFERENCE,TITLE,STATUS,PERFORMACE_DOC,DEADLINE_DOC,FH_DEADLINE,CY_DEADLINE,PERFORMACE_DATE,DEADLINE_DATE
237457,300A,A320,2,242000-22-1,AC GENERATION,O,CS00960026,CS00961320,2123,0,20190115,20190716
237458,300A,A320,2,731000-I1-1,FUEL DISTRIBUTION,O,CS00378884,,6133,0,20180907,20220105
240534,300A,A320,2,ZL-100-02-3,LOWER HALF OF FUSELAGE,O,CS00960026,CS00961320,0,0,20190115,20190516
240535,300A,A320,2,ZL-123-02-1,NOSE LANDING GEAR WELL,O,CS00960026,CS00961320,0,0,20190115,20190516
240536,300A,A320,2,ZL-131-01-1,FORWARD CARGO COMPARTMENT,O,CS00960026,CS00961320,2073,1378,20190115,20190516
240537,300A,A320,2,ZL-147-02-1,MLG-WELL AND HYDRAULIC COMPARTMENT,O,CS00960026,CS00961320,0,0,20190115,20190516
240538,300A,A320,2,ZL-151-01-1,AFT CARGO COMPARTMENT,O,CS00960026,CS00961320,2073,1378,20190115,20190516
240539,300A,A320,2,ZL-161-01-1,BULK CARGO COMPARTMENT,O,CS00960026,CS00961320,2073,0,20190115,20190516
240540,300A,A320,2,ZL-400-02-2,"POWER PLANT, NACELLES AND PYLONS",O,CS00960026,CS00961320,0,1378,20190115,20190516
240541,300A,A320,2,ZL-711-01-1,NOSE GEAR AND NOSE GEAR DOORS,O,CS00960026,CS00961320,0,0,20190115,20190516
